In [89]:
import pandas as pd

rating_data= pd.read_csv('C:/Jupyter_project/datasets/movie_lens/rating.csv')
row_movie_data= pd.read_csv('C:\Jupyter_project\datasets\movie_lens/movie.csv')

In [107]:
"""
parameters : 
    input_data (DataFrame)
        : movie.csv에서 읽은 DataFrame.
          입력된 DataFrame 객체의 원형은 바뀌지 않습니다.
        
    year (integer)
        : year 미만 영화는 거릅니다.
        
    genres (array of string)
        : 분류된 모든 장르가 genres배열에 포함되지 않는 영화는 거릅니다.


return (DataFrame): 
    year 컬럼이 추가되고 조건에 맞게 필터링된 DataFrame
    

*
title이 (year)로 끝나지 않는 15개의 영화는 아예 제외했습니다.
(ex. movield 7789)
       
"""

def movie_filter(input_data, year=1950, genres=[]):
    data = input_data.copy() 
    
    data['year'] = data['title'].str[-5:-1]
    data['year'] = pd.to_numeric(data['year'], downcast='integer', errors='coerce')
    data = data.dropna(axis=0)    
    
    data['genres'] = data['genres'].str.split('|')
    
    data = data[data['year']>=year]
    
    mask = []
    for genre_ in data['genres']:
        mask.append(True if np.intersect1d(genre_, genres).size > 0 else False) 
            
    data = data[mask]
    
    return data

In [108]:
import numpy as np
movie_data = movie_filter(row_movie_data, genres=['Action', 'Crime'])

In [109]:
movie_data.head()

,movieId,title,genres,year
5,6,Heat (1995),"[Action, Crime, Thriller]",1995.0
8,9,Sudden Death (1995),[Action],1995.0
9,10,GoldenEye (1995),"[Action, Adventure, Thriller]",1995.0
14,15,Cutthroat Island (1995),"[Action, Adventure, Romance]",1995.0
15,16,Casino (1995),"[Crime, Drama]",1995.0


In [110]:
def rating_filter(input_data, movie_data, threshold = 100):
    data = input_data.copy()
    
     #movie_data에 없는 평가를 한 경우 뺌
    movie_set = set(movie_data['movieId'].values)
    data = data[data['movieId'].isin(movie_set)]

    #threshold개 이상 평가를 남긴 row만 남김
    x = data['userId'].value_counts()>=threshold
    y = set(x[x].index)
    data = data[data['userId'].isin(y)]
    
    return data

In [111]:
rating_data = rating_filter(rating_data,movie_data)

In [112]:
rating_data.head()

,userId,movieId,rating,timestamp
961,11,10,2.5,2009-01-02 01:15:59
966,11,110,4.0,2009-01-01 23:49:08
967,11,145,3.0,2009-01-01 04:59:07
969,11,153,3.5,2009-01-02 01:15:14
971,11,160,4.0,2009-01-01 05:27:01


In [113]:
R_df = rating_data.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)

In [114]:
def input_user_movie(input_pivot_table):
    copy_pivot = input_pivot_table.copy()
    movie = []
    rating = []
    userID = int(input('사용자의 아이디를 입력하세요 : '))
    size = int(input('평점을 작성할 영화의 개수를 입력하세요 : '))
    for i in range(size) :
        m = int(input ('영화의 ID를 입력하세요. : '))
        movie.append(m)
        r = float(input ('영화의 평점을 입력하세요 : '))
        rating.append(r)
    
    #입력받은 데이터를 하나의 행으로 만들어준다.
    a = pd.DataFrame(data = [rating], columns =movie , index =[userID])
    
    copy_pivot = copy_pivot.append(a).fillna(0)   
    
    return copy_pivot , userID


In [117]:
R_df , userID = input_user_movie(R_df)

사용자의 아이디를 입력하세요 : 138479
평점을 작성할 영화의 개수를 입력하세요 : 5
영화의 ID를 입력하세요. : 9
영화의 평점을 입력하세요 : 3.3
영화의 ID를 입력하세요. : 21
영화의 평점을 입력하세요 : 4.7
영화의 ID를 입력하세요. : 130768
영화의 평점을 입력하세요 : 5
영화의 ID를 입력하세요. : 130842
영화의 평점을 입력하세요 : 2.1
영화의 ID를 입력하세요. : 131011
영화의 평점을 입력하세요 : 1.1


In [118]:
R_df.tail()

,6,9,10,15,16,20,21,22,23,30,...,130768,130836,130842,130984,131011,131025,131056,131104,131122,131180
138467,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138474,5.0,4.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138493,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138479,0.0,3.3,0.0,0.0,0.0,0.0,4.7,0.0,0.0,0.0,...,5.0,0.0,2.1,0.0,1.1,0.0,0.0,0.0,0.0,0.0


In [119]:
from scipy.sparse.linalg import svds
def make_prediction_df():
    
    #R은 pivot_table을 numpy matrix로 만든 것
    R = R_df.values
    
    #user_rating_mean은 사용자의 평균 평점
    user_ratings_mean = np.mean(R, axis = 1)
    
    #R_demeaned : 사용자-영화 테이블에 대해 사용자 평균 평점을 뺀 것
    R_demeaned = R - user_ratings_mean.reshape(-1, 1)
        
    #U 행렬, sigma 행렬, V전치행렬을 반환.
    #이때 spicy에 있는 svd를 이용한다.
    U, sigma, Vt = svds(R_demeaned, k = 50)
    
    #sigma는 0이 포함되지 않은 값으로만 구성되어 있다.
    #sigma를 0이 포함된 대칭 행렬로 변환한다.
    sigma = np.diag(sigma)
    
    #SVD가 적용되어 분해된 R_demeaned를 원본 행렬로 복구
    #구한 원본 행렬에 사용자 평균 rating을 더해준다.
    all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1,1)
    
    preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)
    
    return preds_df


In [120]:
def recommend_movies(predictions_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    
    #인덱스로 변환해주어야 해서 -1 
    user_row_number = userID - 1
    
    #앞에서 만든 prediction행렬을 사용자 인덱스에 따라 영화 정렬 -> 영화 평점이 높은 순으로 정렬
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    #원본 rating_data에서 user_id에 해당하는 데이터를 뽑아낸다.
    user_data = original_ratings_df[original_ratings_df.userId == (userID)]
    
    #user_data와 원본 영화 데이터를 합친다.
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)
                 )
    
    #원본 영화 데이터에서 사용자가 본 영화를 제외한 데이터 추출
    #.merge로 sorted_user_predcitions와 user_full데이터를 합친다.
    #.rename으로 컬럼 이름을 바꾸고 정렬한다
    recommendations = (movies_df[~movies_df['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False)
                      )
                       
    return recommendations


In [121]:
preds_df = make_prediction_df()

In [122]:
prediction= recommend_movies(preds_df,24, movie_data, rating_data, userID)

KeyError: 'movieId'